## Test extract links Wikipedia
Looks we have a limit of 10000 records see [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20test.ipynb) - Test 5 

A try to combine that search with birth year
* result just 10000 records found **lesson learned** is a limiutation of the API


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-10-25 00:02:03.693185


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/latest.php"
url= "https://xn--portrttarkiv-kcb.se/endpoints/search.php"
url= "https://portrattarkiv.se/endpoints/search.php"  


In [3]:
from tqdm.notebook import tqdm  
SPAdetail = "https://portrattarkiv.se/details/"
print(url)  

#print("\n",row["search"],row["item"])
limit = 10000
listdf = []
for year in range(1780,1910,50):
    for start in range(0,10000,limit):
        print("\n\nyear", year,"\tfrom: ",start,"\tlimit",limit,"")
        encoded_body = json.dumps({
            "limit": limit,
            "from":  start,
            "year":  year,
            "facts":{"URL":"https://sv.wikipedia.org/wiki"}
        })
    
        r = http.request('POST', url,
            headers={'Content-Type': 'application/json'},
            body=encoded_body)
        print("http status ", r.status)
        if r.status != 200:
            print("Exit ",r.status)
            print(r)
            break
        
    
        data = json.loads(r.data.decode('utf-8'),) 
        urls = []

        print("\nTotal: ",data["hits"]["total"])
        print(data["_shards"])
        print("Hits: ",len(data["hits"]["hits"]))
        df = pd.json_normalize(data["hits"]["hits"])
        listdf.append(df)

https://portrattarkiv.se/endpoints/search.php


year 1780 	from:  0 	limit 10000 
http status  200

Total:  17187
{'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}
Hits:  10000


year 1830 	from:  0 	limit 10000 
http status  200

Total:  17187
{'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}
Hits:  10000


year 1880 	from:  0 	limit 10000 
http status  200

Total:  17187
{'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}
Hits:  10000


In [4]:
len(listdf)

3

In [5]:
dfTot = pd.concat(listdf) 

In [6]:
#for c in dfTot.columns:
#    print(c)

In [7]:
len(dfTot)

30000

In [8]:
 df = dfTot.drop_duplicates(subset = ["_id"])


In [9]:
len(df[{"_id","_source.Facts.URL"}])

10000

In [10]:
from wikidata.client import Client
from tqdm.notebook import tqdm 

def checkSPA(wikidata_id):
    # get SPA 
    client = Client()
    entity = client.get(wikidata_id, load=True)
    try:
        spa_prop = entity[client.get('P4819')]
    except:
        return False
    return spa_prop
def getWD (url):
    baseUrl = "https://sv.wikipedia.org/w/api.php?action=query&prop=pageprops|wbentityusage&titles="
    title = url.replace("https://sv.wikipedia.org/wiki/","")
    urlWD = baseUrl + title
    wdr = http.request('Get', urlWD + "&format=json",
                 headers={'Content-Type': 'application/json'})

    if wdr.status != 200:
        print("Error ", wdr.status)
        return False
    
    datawd = json.loads(wdr.data.decode('utf-8'),) 
    pg_dict = datawd['query']['pages']
    pg_key = list(pg_dict.keys())[0]
    wikidata_id = pg_dict[pg_key]['pageprops']['wikibase_item']
    return wikidata_id

for index, row in tqdm( df.iterrows(),total= df.shape[0]):
    #print(row['_id'], row['_source.Facts.URL'])
    for n in row['_source.Facts.URL']:
        if "wikipedia" in n.lower():
            try: 
                wdrec = getWD(n)
                if checkSPA(wdrec) == False:
                    print ("Todo",row['_id'],wdrec,n)
                #print (n,"Value :",getWD(n))
            except:
                pass
  

  0%|          | 0/10000 [00:00<?, ?it/s]

Todo sj9PGLAlnmUAAAAAACaEqg Q61992416 https://sv.wikipedia.org/wiki/Fannyudde
Todo seuNpYu6DKAAAAAAAABhOA Q16158940 https://sv.wikipedia.org/wiki/Thorsten_Arwidsson_(kartograf)
Todo seuNpYu6DKAAAAAAAABhOA Q16158940 https://sv.wikipedia.org/wiki/Thorsten_Arwidsson_(kartograf)
Todo seuNpYu6DKAAAAAAAABjcQ Q5923884 https://sv.wikipedia.org/wiki/Thorgny_O.B.N._Krok
Todo seuNpYu6DKAAAAAAAABjcQ Q5923884 https://sv.wikipedia.org/wiki/Thorgny_O.B.N._Krok
Todo 28K7pO-SRrAAAAAAAACqzg Q5548345 https://sv.wikipedia.org/wiki/Hjalmar_Alving
Todo TNQaWo324IAAAAAAAADUzw Q26240497 https://sv.wikipedia.org/wiki/Einar_Bergwall
Todo sj9PGLAlnmUAAAAAAB2U_w Q747922 https://sv.wikipedia.org/wiki/John_Erhard_Areschoug
Todo sj9PGLAlnmUAAAAAABJ7ig Q24680604 https://sv.wikipedia.org/wiki/Helmer_Bratt
Todo TNQaWo324IAAAAAAAAC0ZQ Q656239 https://sv.wikipedia.org/wiki/Per_Gustaf_Berg
Todo WQknIBFpuuAAAAAAAAN7jQ Q1353945 https://sv.wikipedia.org/wiki/Erik_Gustaf_Lidbeck
Todo rkIpfGqHXrAAAAAAAACSyA Q97483532 https://s

Todo sj9PGLAlnmUAAAAAAA1tTQ Q5577360 https://sv.wikipedia.org/wiki/Adolf_Leopold_Bj%C3%B6rck
Todo rkIpfGqHXrAAAAAAAABm6Q Q4802430 https://sv.wikipedia.org/wiki/Mauritz_Lindstr%C3%B6m
Todo rkIpfGqHXrAAAAAAAAD3Cg Q5956089 https://sv.wikipedia.org/wiki/John_Lindgren_(historiker)
Todo sj9PGLAlnmUAAAAAACYyRg Q16158211 https://sv.wikipedia.org/wiki/Anna-Lisa_Annell
Todo I_2wtd5VefAAAAAAAABLGA Q274489 https://sv.wikipedia.org/wiki/Yngsj%C3%B6mordet
Todo sj9PGLAlnmUAAAAAACd8mQ Q471307 https://sv.wikipedia.org/wiki/Karl-August_Fagerholm
Todo sj9PGLAlnmUAAAAAABU-RQ Q4992736 https://sv.wikipedia.org/wiki/Carl_Wilhelm_B%C3%B6ttiger
Todo wbbNj8LLWMAAAAAAAAAwXw Q27243339 https://sv.wikipedia.org/wiki/Nisser_(sl%C3%A4kt)
Todo sj9PGLAlnmUAAAAAACZUTA Q16498578 https://sv.wikipedia.org/wiki/Marie-Anne_Bergman-Larby
Todo sj9PGLAlnmUAAAAAABNsOw Q5578133 https://sv.wikipedia.org/wiki/Frey_Bj%C3%B6rlingson
Todo sj9PGLAlnmUAAAAAAA3-Cw Q5563972 https://sv.wikipedia.org/wiki/G%C3%B6sta_Bagge
Todo j6S8rJvx9lAAA

### Next step 
* Use [Open Refine](https://www.wikidata.org/wiki/Wikidata:Tools/OpenRefine) to set Wikidata [Property:P4819](https://www.wikidata.org/wiki/Property:P4819) 
* see video in Swedish "[SPA API find same as Wikipedia and use OpenRefine . in Swedish](https://youtu.be/bTWtG3NA0uo)"
